In [1]:
# Generate the video for the env with a trained model

In [3]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv

import torchdriveenv
from torchdriveenv.env_utils import load_default_env_config, load_default_validation_data

/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Commercial access denied and fallback to check for academic access.


In [4]:
model = SAC.load("model.zip")

/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 73.74GB > 49.74GB
  warnings.warn(


In [5]:
data = load_default_validation_data()
env_config = load_default_env_config()
env_config.render_mode="video"

In [6]:
def make_env():
    env = gym.make('torchdriveenv-v0', args={'cfg': env_config, 'data': data})
    env = Monitor(env)  
    return env

In [7]:
def main(cfg, model):
    env = DummyVecEnv([make_env])
    env = VecFrameStack(env, n_stack=3, channels_order="first")
    obs = np.expand_dims(env.reset()[0], 0)
    states = None
    episode_starts = [True]

    while True:
        actions, states = model.predict(
         obs,  
         state=states,
         episode_start=episode_starts,
         deterministic=True,
        )
        obs, reward, done, info = env.step(actions)      
        if done:
            break
    env.close()

In [8]:
env_config

EnvConfig(ego_only=False, max_environment_steps=200, use_background_traffic=True, terminated_at_infraction=True, seed=None, simulator=TorchDriveConfig(renderer=RendererConfig(backend='default', render_agent_direction=True, left_handed_coordinates=True, highlight_ego_vehicle=True), single_agent_rendering=False, collision_metric=<CollisionMetric.nograd: 'nograd'>, offroad_threshold=0.5, left_handed_coordinates=True), render_mode='video', video_filename='rendered_video.mp4', video_res=1024, video_fov=500, device='cuda')

In [9]:
main(env_config, model)

INFO:torchdriveenv.gym_env:seed: 524755634
INFO:torchdriveenv.gym_env:    def get_reward(self):
        x = self.simulator.get_state()[..., 0]
        y = self.simulator.get_state()[..., 1]
        psi = self.simulator.get_state()[..., 2]

        d = math.dist((x, y), (self.last_x, self.last_y)) if (self.last_x is not None) and (self.last_y is not None) else 0
        distance_reward = 1 if d > 0.5 else 0
        psi_reward = (1 - math.cos(psi - self.last_psi)) * (-20.0) if (self.last_psi is not None) else 0
        if self.check_reach_target():
            reach_target_reward = 10
            self.reached_waypoint_num += 1
        else:
            reach_target_reward = 0
        r = torch.zeros_like(x)
        r += reach_target_reward + distance_reward + psi_reward
        return r.item()

